# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

- skip 1 month and 3 month returns
- filter by 10 day avg volume > 100,000

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime

In [2]:
import os

In [3]:
import api_token  as gwg     # handle iexcloud API tokens

env = "cloud" # "sandbox"  # 
BASE_URL, IEX_CLOUD_API_TOKEN = gwg.get_base_url(env=env), gwg.get_api_token(env=env)

In [4]:
WRITE_ALL_DF = False

TOP_N = 100
portfolio_size = 200000
filename = "iexcloud"
MIN_AVG_VOL = 100000    # 100k shares
MIN_PRICE = 2.0         # over $2

In [5]:
today = datetime.datetime.now()

today_str = today.strftime('%Y-%m-%d')
__file__ = "/home/wengong/projects/algorithmic-trading-python/mom101/momentum_rank.py"
pardir = os.path.dirname(__file__)
scan_dir = os.path.join(pardir, "scan", today_str)
if not os.path.exists(scan_dir):
    os.mkdir(scan_dir)
    


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [6]:
api_url = f'{BASE_URL}/stable/ref-data/iex/symbols?token={IEX_CLOUD_API_TOKEN}'
api_url
data = requests.get(api_url).json()

In [7]:
stocks = [d['symbol']  for d in data if d['isEnabled'] 
           and "." not in d['symbol'] 
           and "-" not in d['symbol'] 
           and "+" not in d['symbol']
           and "=" not in d['symbol']
          ]

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 5-day price returns
* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [9]:
hqm_columns = [
    'Ticker', 
    'Name',
    'Number of Shares to Buy', 
    'Price', 
    'Avg10Volume',
    '5day Price Return',
    '1mon Price Return',
    '3mon Price Return',
    '6mon Price Return',
    '12mon Price Return', 
    '5day Return Percentile',
    '1mon Return Percentile',
    '3mon Return Percentile',
    '6mon Return Percentile',
    '12mon Return Percentile',
    'HQM Score',
    'ma50',
    'ma200'
]

    
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
ibatch = 0
for symbol_string in symbol_strings:
    ibatch += 1
    print(f"ibatch = {ibatch}")
    # print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                            pd.Series([symbol, 
                                       data[symbol]['stats']['companyName'],
                                       '',
                                       data[symbol]['quote']['latestPrice'],
                                       data[symbol]['stats']['avg10Volume'],
                                       data[symbol]['stats']['day5ChangePercent'],
                                       data[symbol]['stats']['month1ChangePercent'],
                                       data[symbol]['stats']['month3ChangePercent'],
                                       data[symbol]['stats']['month6ChangePercent'],
                                       data[symbol]['stats']['year1ChangePercent'],
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       'N/A',
                                       data[symbol]['stats']['day50MovingAvg'],
                                       data[symbol]['stats']['day200MovingAvg']
                                       ], 
                                      index = hqm_columns), 
                            ignore_index = True)
        except Exception as ex:
            print(f"symbol = {symbol}\n Error={str(ex)}")
        
hqm_dataframe.columns

ibatch = 1
ibatch = 2
ibatch = 3
ibatch = 4
ibatch = 5
ibatch = 6
ibatch = 7
ibatch = 8
ibatch = 9
ibatch = 10
ibatch = 11
ibatch = 12
ibatch = 13
symbol = CBO
 Error='CBO'
symbol = CBX
 Error='CBX'
ibatch = 14
ibatch = 15
ibatch = 16
ibatch = 17
ibatch = 18
ibatch = 19
ibatch = 20
ibatch = 21
ibatch = 22
ibatch = 23
ibatch = 24
ibatch = 25
ibatch = 26
ibatch = 27
ibatch = 28
ibatch = 29
ibatch = 30
ibatch = 31
ibatch = 32
ibatch = 33
ibatch = 34
ibatch = 35
ibatch = 36
ibatch = 37
ibatch = 38
ibatch = 39
ibatch = 40
ibatch = 41
ibatch = 42
ibatch = 43
ibatch = 44
ibatch = 45
ibatch = 46
ibatch = 47
ibatch = 48
ibatch = 49
ibatch = 50
ibatch = 51
ibatch = 52
ibatch = 53
ibatch = 54
ibatch = 55
ibatch = 56
symbol = NTEST
 Error='NTEST'
ibatch = 57
ibatch = 58
ibatch = 59
ibatch = 60
ibatch = 61
ibatch = 62
ibatch = 63
ibatch = 64
ibatch = 65
ibatch = 66
ibatch = 67
ibatch = 68
ibatch = 69
ibatch = 70
ibatch = 71
ibatch = 72
ibatch = 73
ibatch = 74
ibatch = 75
ibatch = 76
ibatch = 77
iba

Index(['Ticker', 'Name', 'Number of Shares to Buy', 'Price', 'Avg10Volume',
       '5day Price Return', '1mon Price Return', '3mon Price Return',
       '6mon Price Return', '12mon Price Return', '5day Return Percentile',
       '1mon Return Percentile', '3mon Return Percentile',
       '6mon Return Percentile', '12mon Return Percentile', 'HQM Score',
       'ma50', 'ma200'],
      dtype='object')

In [11]:
hqm_dataframe

,Ticker,Name,Number of Shares to Buy,Price,Avg10Volume,5day Price Return,1mon Price Return,3mon Price Return,6mon Price Return,12mon Price Return,5day Return Percentile,1mon Return Percentile,3mon Return Percentile,6mon Return Percentile,12mon Return Percentile,HQM Score,ma50,ma200
0,A,Agilent Technologies Inc.,,117.31,1919967,-0.0166806,0.0258854,0.203004,0.370993,0.384638,N/A,N/A,N/A,N/A,N/A,N/A,114.33,101.46
1,AA,Alcoa Corp,,21.96,3552001,-0.00227169,0.078056,0.901299,1.00915,0.0237762,N/A,N/A,N/A,N/A,N/A,N/A,20.52,14.80
2,AAA,Listed Funds Trust,,25.09,2013,0.000398724,0.00247723,0.0067168,0.00310647,0.00310647,N/A,N/A,N/A,N/A,N/A,N/A,25.03,24.95
3,AAAU,Goldman Sachs Physical Gold ETF Shares,,18.71,381027,-0.000534188,0.0406007,0.00753904,0.0588568,0.241375,N/A,N/A,N/A,N/A,N/A,N/A,18.44,18.74
4,AACG,ATA Creativity Global,,1.2799,32577,0.0664056,0.00779528,0.163545,0.112957,-0.131506,N/A,N/A,N/A,N/A,N/A,N/A,1.19,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8646,ZVO,Zovio Inc,,4.05,151622,-0.00246305,0.100543,0,0.40625,0.910377,N/A,N/A,N/A,N/A,N/A,N/A,4.01,4.10
8647,ZVZZT,,,10.03,0,None,None,None,None,None,N/A,N/A,N/A,N/A,N/A,N/A,0.00,0.00
8648,ZYME,Zymeworks Inc,,50.13,236145,-0.0262238,0.0325438,0.109807,0.391727,0.113011,N/A,N/A,N/A,N/A,N/A,N/A,49.02,39.79
8649,ZYNE,Zynerba Pharmaceuticals Inc,,3.42,1527599,0.00588235,-0.253275,0.0426829,-0.457143,-0.417376,N/A,N/A,N/A,N/A,N/A,N/A,4.04,3.94


In [13]:
hqm_dataframe.shape
# (8651, 18)

(8651, 18)

In [14]:
time_periods = [
    '5day',
    '1mon',
    '3mon',
    '6mon',
    '12mon'
]
price_return_cols = [f'{time_period} Price Return' for time_period in time_periods]

price_return_cols

['5day Price Return',
 '1mon Price Return',
 '3mon Price Return',
 '6mon Price Return',
 '12mon Price Return']

In [15]:
hqm_dataframe = hqm_dataframe.dropna(
        how='any', 
        subset=price_return_cols+["Price"]
)

In [16]:
hqm_dataframe[hqm_dataframe["12mon Price Return"].isnull()]

,Ticker,Name,Number of Shares to Buy,Price,Avg10Volume,5day Price Return,1mon Price Return,3mon Price Return,6mon Price Return,12mon Price Return,5day Return Percentile,1mon Return Percentile,3mon Return Percentile,6mon Return Percentile,12mon Return Percentile,HQM Score,ma50,ma200


In [17]:
# apply filters
hqm_dataframe = hqm_dataframe[
        (hqm_dataframe["Avg10Volume"] >= MIN_AVG_VOL) & 
        (hqm_dataframe["Price"] >= MIN_PRICE) & 
        (hqm_dataframe["ma50"] > hqm_dataframe["ma200"])
    ]

In [18]:
hqm_dataframe.shape

(3768, 18)

In [19]:
import pickle
# backup df
with open(f"df_hqm_{today_str}.pkl", "wb") as f:
    pickle.dump(hqm_dataframe, f)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

Here's how we'll do this:

In [20]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Name,Number of Shares to Buy,Price,Avg10Volume,5day Price Return,1mon Price Return,3mon Price Return,6mon Price Return,12mon Price Return,5day Return Percentile,1mon Return Percentile,3mon Return Percentile,6mon Return Percentile,12mon Return Percentile,HQM Score,ma50,ma200
0,A,Agilent Technologies Inc.,,117.31,1919967,-0.0166806,0.0258854,0.203004,0.370993,0.384638,0.200106,0.423036,0.362261,0.532378,0.718153,N/A,114.33,101.46
1,AA,Alcoa Corp,,21.96,3552001,-0.00227169,0.078056,0.901299,1.00915,0.0237762,0.385881,0.653662,0.90897,0.870754,0.391985,N/A,20.52,14.80
5,AACQ,Artius Acquisition Inc,,10.46,912225,0.00287632,0.0544355,0.0587045,0.046,0.046,0.5069,0.575106,0.106688,0.0764331,0.431263,N/A,10.07,9.93
7,AACQW,Artius Acquisition Inc,,2.16,411850,0.270588,0.52102,0.479553,0.234286,0.234286,0.978238,0.954618,0.724788,0.32776,0.643843,N/A,1.55,1.47
9,AAIC,Arlington Asset Investment Corp,,3.84,288830,0.026738,0.160121,0.416974,0.347368,-0.327813,0.761412,0.807059,0.663217,0.500531,0.0806794,N/A,3.38,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8643,ZTS,Zoetis Inc,,160.72,1564272,-0.017784,0.00330857,0.000504859,0.214734,0.21507,0.189225,0.274947,0.0257431,0.295117,0.631369,N/A,161.78,155.49
8644,ZUMZ,Zumiez Inc,,36.01,309737,-0.0501187,-0.0212014,0.325847,0.389275,0.0866023,0.0501592,0.167861,0.550955,0.55414,0.493365,N/A,36.18,29.43
8645,ZUO,Zuora Inc,,14.36,1639423,0.129819,0.22735,0.372849,0.10717,-0.000695894,0.944268,0.865977,0.613854,0.141454,0.360403,N/A,11.79,11.55
8648,ZYME,Zymeworks Inc,,50.13,236145,-0.0262238,0.0325438,0.109807,0.391727,0.113011,0.130573,0.467357,0.18551,0.554936,0.529989,N/A,49.02,39.79


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

# Selecting Stocks based on different weights

As before, we can identify the 100 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 100 entries.

In [21]:
!pwd

/home/wengong/projects/algorithmic-trading-python/finished_files


In [22]:
time_period_idx_map = {}
for i,t in enumerate(time_periods):
    time_period_idx_map[t] = i

In [23]:
def weighted_mean(mom_percentiles, tp_weight_map):
    """ Perform weighted mean for HQM score
    """
    weight = 0
    for k,v in tp_weight_map.items():
        if v >= 0:
            weight += v * mom_percentiles[time_period_idx_map[k]] 
        else:
            weight += v * (mom_percentiles[time_period_idx_map[k]] - 1)
    return weight

def tp_weight_name(tp_weight_map):
    name = ""
    for k,v in tp_weight_map.items():
        name += f"_{k}_{v}"
    return name

In [40]:

# change weights for different selection criteria

# case-1 : up-up-up-up-up
tp_weight_map = {
    '5day' : 0.2,
    '1mon' : 0.2,
    '3mon': 0.2,
    '6mon' : 0.2,
    '12mon' : 0.2  
}

# case-2 : up-up-up-up-up
tp_weight_map = {
    '5day' : 0.1,
    '1mon' : 0.1,
    '3mon': 0.2,
    '6mon' : 0.3,
    '12mon' : 0.3  
}

# case-3 : down-down-up-up-up
tp_weight_map = {
    '5day' : -0.1,
    '1mon' : -0.1,
    '3mon': 0.2,
    '6mon' : 0.3,
    '12mon' : 0.3  
}

In [41]:
suffix = tp_weight_name(tp_weight_map)
xlsx_file = os.path.join(scan_dir, f"ss_{filename}_top{TOP_N}{suffix}.xlsx")
xlsx_file_all = os.path.join(scan_dir, f"ss_{filename}_all{suffix}.xlsx")
watchlist_file = os.path.join(scan_dir, f"wl_{filename}_top{TOP_N}{suffix}.csv")

# from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    # hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    hqm_dataframe.loc[row, 'HQM Score'] = weighted_mean(momentum_percentiles, tp_weight_map)

# sort by HQM score
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)

if WRITE_ALL_DF:
    hqm_dataframe_all = hqm_dataframe.copy()

df_hqm = hqm_dataframe[:(TOP_N+1)]

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [42]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')

In [43]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [44]:
column_formats = { 
        'A': ['Ticker', string_template],
        'B': ['Name', string_template], 
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['Price', dollar_template],
        'E': ['Avg10Volume', integer_template],
        'F': ['5day Price Return', percent_template], 
        'G': ['1mon Price Return', percent_template],
        'G': ['3mon Price Return', percent_template],
        'G': ['6mon Price Return', percent_template],
        'H': ['12mon Price Return', percent_template],
        'I': ['5day Return Percentile', percent_template],
        'J': ['1mon Return Percentile', percent_template],
        'J': ['3mon Return Percentile', percent_template],
        'J': ['6mon Return Percentile', percent_template],
        'K': ['12mon Return Percentile', percent_template], 
        'L': ['HQM Score', percent_template],
        'M': ['ma50', dollar_template],
        'N': ['ma200', dollar_template]
}

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [45]:
df_hqm.to_excel(writer, sheet_name='Momentum Strategy', index = False)

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

writer.save()

In [46]:
if WRITE_ALL_DF:
    writer = pd.ExcelWriter(xlsx_file2, engine='xlsxwriter')
    hqm_dataframe_all.to_excel(writer, sheet_name='Momentum Strategy', index = False)

    for column in column_formats.keys():
        writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
        writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

    writer.save()

### save watchlist

In [47]:
tickers = df_hqm[['Ticker']]
tickers.to_csv(watchlist_file, index=False)